# Import Libraries
Also imported the csv file for user and reviews. Stored it in dfUserReviews 

In [1]:
import csv
import ast
import json
import pandas as pd

dfUserReviews=pd.read_csv('australian_user_reviews_output.csv')
dfUserReviews.head()
dfUserReviews.describe()
dfUserReviews.shape

(17950, 7)

# Import csv files
Imported the csv file for user and items and store it in dfUserItems

In [2]:
dfUserItems=pd.read_csv('australian_users_items_output.csv')
dfUserItems.head()
dfUserItems.describe()
dfUserItems.shape

(85876, 6)

# Merge
Merge the previous df into one for further use

In [3]:
mergedStuff = pd.merge(dfUserReviews, dfUserItems, on=['item_id','user_id'], how='outer')
mergedStuff.head(10)

,user_id,user_url_x,funny,posted,last_edited,item_id,recommend,user_url_y,items_count,item_name,playtime_forever
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250,True,http://steamcommunity.com/profiles/76561197970...,277.0,Killing Floor,10006.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200,True,http://steamcommunity.com/profiles/76561197970...,277.0,Zeno Clash,271.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110,True,http://steamcommunity.com/profiles/76561197970...,277.0,Metro 2033,834.0
3,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",NaN,211420,True,NaN,NaN,NaN,NaN
4,maplemage,http://steamcommunity.com/id/maplemage,1 person found this review funny,"Posted December 23, 2013.",NaN,211820,True,NaN,NaN,NaN,NaN
5,maplemage,http://steamcommunity.com/id/maplemage,2 people found this review funny,"Posted March 14, 2014.",NaN,730,True,NaN,NaN,NaN,NaN
6,maplemage,http://steamcommunity.com/id/maplemage,NaN,"Posted July 11, 2013.",NaN,204300,True,NaN,NaN,NaN,NaN
7,76561198156664158,http://steamcommunity.com/profiles/76561198156...,NaN,Posted June 16.,NaN,252950,True,NaN,NaN,NaN,NaN
8,WeiEDKrSat,http://steamcommunity.com/id/WeiEDKrSat,NaN,"Posted December 25, 2013.",NaN,4000,True,NaN,NaN,NaN,NaN
9,death-hunter,http://steamcommunity.com/id/death-hunter,NaN,"Posted September 5, 2015.",NaN,232090,True,NaN,NaN,NaN,NaN


# Create New DF
Create a new dataframe which has only user_id, item_id and recommend values

In [4]:
df=mergedStuff[['user_id','item_id','recommend']]

In [5]:
df.head()
df.dtypes

user_id      object
item_id       int64
recommend    object
dtype: object

# Drop Null Values
Keep only those values which are required for training and testing. Thr NaN values are dropped

In [6]:
dfTF=df.dropna()

In [7]:
dfTF['recommend']=dfTF['recommend'].astype(int)

C:\Users\ketan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Import libraries for Model Creation

In [8]:
import pandas as pd
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import ShuffleSplit
import csv

In [9]:
reader = Reader(rating_scale=(0,1))
data=Dataset.load_from_df(dfTF,reader)

# Use ShuffleSplit for random shuffling of test and train dataset

In [10]:
sp=ShuffleSplit(n_splits=20,random_state=10)

# SVD Algorithm

In [11]:
algo = SVD(n_factors=250,n_epochs=15,lr_all=0.03,reg_qi=0.080,random_state=20,init_std_dev=0.01)

In [16]:
import time

start_time=time.time()
for trainset, testset in sp.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

time_required=time.time()-start_time
print("TIme required for SVD: ",time_required)

RMSE: 0.3017
RMSE: 0.2967
RMSE: 0.3062
RMSE: 0.3095
RMSE: 0.2912
RMSE: 0.3058
RMSE: 0.2966
RMSE: 0.3034
RMSE: 0.2978
RMSE: 0.2988
RMSE: 0.3190
RMSE: 0.3064
RMSE: 0.3017
RMSE: 0.2985
RMSE: 0.3046
RMSE: 0.3098
RMSE: 0.3080
RMSE: 0.3040
RMSE: 0.3062
RMSE: 0.2932
TIme required for SVD:  23.67420506477356


# KNN Algorithm

In [17]:
sim_options = {
    'name': 'pearson', #similarity measure
    'user_based': False #item based
}

In [19]:
knn = KNNBasic(sim_options=sim_options,k=3,min_k=1) #neighbours=3

In [20]:
import time

start_time=time.time()
for trainset, testset in sp.split(data):

    # train and test algorithm.
    knn.fit(trainset)
    predictionsKNN = knn.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictionsKNN, verbose=True)

time_required=time.time()-start_time
print("TIme required for KNN: ",time_required)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3181
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3154
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3245
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3247
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3128
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3226
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3160
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3201
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3226
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3170
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.3353
Computing 

# Baseline Algorithm

In [21]:
trainset, testset = train_test_split(data, test_size=.25)

In [22]:
bsl_options = {'method': 'sgd',#another option is als
               'n_epochs': 40, # number of iterations
               'learning_rate': 0.007, 
               'reg_i': 20, # item-regularisation parameter
               'reg_u': 35 # user-regularisation parameter
               }

In [25]:
start_time=time.time()

bl = BaselineOnly(bsl_options=bsl_options) #neighbours=3
bl.fit(trainset)
predictions_bl = bl.test(testset)
accuracy.rmse(predictions_bl)

time_required=time.time()-start_time
print("TIme required for BaseLine: ",time_required)

Estimating biases using sgd...
RMSE: 0.3085
TIme required for BaseLine:  0.1361675262451172
